In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('train.csv')
print(df.shape)
df.head()

(40000, 2)


,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos


In [3]:
print(df.iloc[11]['review'])

Robert Jordan is a television star. Robert Jordan likes things orderly, on time and properly executed. In his world children are to be seen, not heard. So why would Mr. Jordan want to become the master of a rambunctious band of Boy Scouts? Ratings. His staff figures that if learns how to interact with the youth, they will be more inclined to watch his show. Of course watching Jordan cope comprises most of the fun.<br /><br />Like Mr. Belvedere and Mr. Belvedere Goes to College this one is sure to please.<br /><br />ANYONE INTERESTED IN OBTAINING A COPY OF THIS FILM PLEASE WRITE TO ME AT: IAMASEAL2@YAHOO.COM


In [4]:
df.isnull().sum()

review    0
label     0
dtype: int64

In [5]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [6]:
sw=set(stopwords.words('english'))
ps=PorterStemmer()

In [7]:
def clean(sample):
    sample=sample.lower()
    sample=sample.replace('<br /><br />','')
    sample=re.sub('[^a-zA-Z]+',' ',sample)
    sample=sample.split()
    
    sample=[ps.stem(s) for s in sample if s not in sw]
    sample=" ".join(sample)
    return sample

In [8]:
spl=df.iloc[11]['review']

In [9]:
clean(spl)

'robert jordan televis star robert jordan like thing orderli time properli execut world children seen heard would mr jordan want becom master rambuncti band boy scout rate staff figur learn interact youth inclin watch show cours watch jordan cope compris fun like mr belveder mr belveder goe colleg one sure pleas anyon interest obtain copi film pleas write iamas yahoo com'

In [10]:
df['clean_rvw']=df['review'].apply(clean)

In [11]:
df.head()

,review,label,clean_rvw
0,mature intelligent and highly charged melodram...,pos,matur intellig highli charg melodrama unbelive...
1,http://video.google.com/videoplay?docid=211772...,pos,http video googl com videoplay docid hl en dis...
2,Title: Opera (1987) Director: Dario Argento Ca...,pos,titl opera director dario argento cast cristin...
3,I think a lot of people just wrote this off as...,pos,think lot peopl wrote anoth one tom cruis weir...
4,This is a story of two dogs and a cat looking ...,pos,stori two dog cat look way back home old wise ...


In [12]:
corpus=df['clean_rvw'].values

In [13]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [30]:
cv=CountVectorizer(max_df=0.5,max_features=10000)

In [31]:
X=cv.fit_transform(corpus)

In [32]:
X.shape

(40000, 10000)

In [33]:
tfidf=TfidfTransformer()

In [34]:
X=tfidf.fit_transform(X)

In [35]:
X.shape

(40000, 10000)

In [36]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=df['label']
Y=le.fit_transform(Y)

In [37]:
Y=Y.astype('int')

In [53]:
x_val=X[:5000]
y_val=Y[:5000]
x_t=X[5000:]
y_t=Y[5000:]


In [54]:
from keras import models
from keras.layers import Dense

In [55]:
model=models.Sequential()

In [56]:
model.add(Dense(16,activation='relu',input_shape=(X.shape[1],)))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [59]:
print(y_val.shape)
print(y_t.shape)

(5000,)
(35000,)


In [60]:
hist=model.fit(x_t,y_t,batch_size=128,epochs=5,validation_data=(x_val,y_val))

InvalidArgumentError: indices[1] = [0,9893] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]